In [ ]:
# Notebook imports

# Used to define paths to files for data import and export
import os
import os.path as op

# Basic libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns 

# Used for preprocessing and splitting of data 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Different models for feature selection
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector

# Different models to be trained later, can be used as classifiers for feature selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb

# Feature selection notebook

### Part 1 : Loading preprocessed data

### Part 2 : Splitting in train and test sets

We need train and test sets to be able to do feature selection using the sklearn models.

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

### Part 3 : Data preprocessing

In [ ]:
# Define the categorical features
categorical = []

# Preprocess data
# 1. Separate in categorical and continuous columns
df_train_cat = df_train[df_train.columns[df_train.columns.isin(categorical)]]
df_train_con = df_train[df_train.columns[~df_train.columns.isin(categorical)]]
df_test_cat = df_test[df_test.columns[df_test.columns.isin(categorical)]]
df_test_con = df_test[df_test.columns[~df_test.columns.isin(categorical)]]

cols_cat = df_train_cat.columns
cols_con = df_train_con.columns

# 2. Fill the missing values if any, no missing values accepted by the models

# For categorical variables
imp_cat = SimpleImputer(strategy="most_frequent")
imp_cat = imp_cat.fit(df_train_cat)

df_train_cat = imp_cat.transform(df_train_cat)
df_test_cat = imp_cat.transform(df_test_cat)
df_train_cat = pd.DataFrame(df_train_cat, columns=cols_cat)
df_test_cat = pd.DataFrame(df_test_cat, columns=cols_cat)

# For continuous variables
imp_con = SimpleImputer(strategy="median")
imp_con = imp_con.fit(df_train_con)

df_train_con = imp_con.transform(df_train_con)
df_test_con = imp_con.transform(df_test_con)
df_train_con = pd.DataFrame(df_train_con, columns=cols_con)
df_test_con = pd.DataFrame(df_test_con, columns=cols_con)

# 3. Scale the continuous variables

scaler = StandardScaler()
scaler = scaler.fit(df_train_con)

df_train_con = scaler.transform(df_train_con)
df_test_con = scaler.transform(df_test_con)
df_train_con = pd.DataFrame(df_train_con, columns=cols_con)
df_test_con = pd.DataFrame(df_test_con, columns=cols_con)

# 4. Merge the categorical and continuous variables
df_train = pd.concat([df_train_cat, df_train_con], axis=1)
df_test = pd.concat([df_test_cat, df_test_con], axis=1)

# 5. Get X and y matrices for feature classification
X_train = df_train.drop(columns=[""])
y_train = df_train[""]
X_test = df_test.drop(columns=[""])
y_test = df_test[""]

# 6. Get feature names
features = X_train.columns

### Part 4 : Feature selection
#### 4.1 Choosing classifier

In [ ]:
# Choosing the classifier
classifier = LogisticRegression()
name_classifier = 'LogisticRegression' # give the name of the classifier (str) to save the file at the end

# Choosing the minimum number of features so consider
min_features_to_select = 15 

# List of other possible classifiers :
    # LogisticRegression()
    # KNeighborsClassifier()
    # svm.SVC()
    # MLPClassifier()
    # QuadraticDiscriminantAnalysis()
    # xgb.XGBClassifier(objective="binary:logistic", random_state=42)

#### 4.2 Recursive feature elimination (RFE) with cross-validation

In [ ]:
# Create the RFE object and compute a cross-validated score
classifier = LogisticRegression(max_iter = 500)

rfecv = RFECV(estimator=classifier,
              step=1,
              cv=StratifiedKFold(4),
              scoring='f1',
              min_features_to_select=min_features_to_select)
rfecv.fit(X_train, y_train)

In [ ]:
# Optimal number of features
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (f1_score)")
plt.plot(range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

In [ ]:
# See feature ranking
df_rfecv = pd.DataFrame(list(zip(features, rfecv.ranking_)), columns =['Feature', 'Rank']).sort_values(by='Rank')
df_rfecv.head(40)

In [ ]:
# Columns selected
features_rfecv = features[rfecv.support_]
features_rfecv

#### 4.3 Sequential Feature Selection
##### 4.3.1 SelectFromModel

First we get an idea of the importance of the features with a logistic regression. Features with the highest absolute coefficient are considered most important

In [ ]:
# Feature importance from coefficients
classifier = LogisticRegression(max_iter = 500)
importance_fit = classifier.fit(X_train, y_train)

if len(importance_fit.coef_.tolist())>1:
       importance = np.abs(importance_fit.coef_)  
else:
       importance = np.abs(importance_fit.coef_.tolist()[0])

df_importance = pd.DataFrame(list(zip(features, importance)), columns =['Feature', 'Importance']).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(25, 7))
plt.bar('Feature', 'Importance', data=df_importance)
plt.title("Feature importances via coefficients")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# See feature ranking
df_importance.head(10)

Now we select the features which are most important according to the coefficients using SelectFromModel.

In [ ]:
threshold = np.sort(importance)[-35] + 0.01
sfm = SelectFromModel(classifier, threshold=threshold).fit(X_train, y_train)

features_sfm = features[sfm.get_support()]
print(f"Features selected by SelectFromModel: {features_sfm}")

##### 4.3.2 SequentialFeatureSelection

In [ ]:
# Forward sequential feature selection
classifier = LogisticRegression(max_iter = 500)
sfs_forward = SequentialFeatureSelector(classifier, n_features_to_select=20, direction="forward").fit(X_train, y_

In [ ]:
# Backwards sequential feature selection, takes a lot of time to run
classifier = LogisticRegression(max_iter = 500)
sfs_backward = SequentialFeatureSelector(classifier, n_features_to_select=20, direction="backward").fit(X_train

In [ ]:
# Selected features
features_sfs_forward = features[sfs_forward.get_support()]
print("Features selected by forward sequential selection:\n " f"{features_sfs_forward}")

#print('------')

features_sfs_backward = features[sfs_backward.get_support()]
print("Features selected by backward sequential selection:\n " f"{features_sfs_backward}")

## 5. Discussion